<a href="https://colab.research.google.com/github/LaJeremi/Tensorflow-nlp-tutorial-Practice-/blob/main/13.%20Subword%20Tokenizer/%2013_04_Huggingface_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 13-04 허깅페이스 토크나이저(Huggingface Tokenizer)


### 자연어 처리 스타트업 허깅페이스가 개발한 패키지 tokenizers는 자주 등장하는 서브워드들을 하나의 토큰으로 취급하는 다양한 서브워드 토크나이저를 제공


# 1. BERT의 워드피스 토크나이저(BertWordPieceTokenizer)

구글이 공개한 딥 러닝 모델 BERT에는 WordPiece Tokenizer가 사용

여기서는 네이버 영화 리뷰 데이터를 해당 토크나이저에 학습시키고, 이로부터 서브워드의 단어 집합(Vocabulary)을 얻습니다. 그리고 임의의 문장에 대해서 학습된 토크나이저를 사용하여 토큰화를 진행합니다. 우선 네이버 영화 리뷰 데이터를 로드



In [ ]:
pip install tokenizers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 32.4 MB/s eta 0:00:00


In [ ]:
import tokenizers
tokenizers.__version__

'0.13.2'

In [ ]:

import pandas as pd
import urllib.request
from tokenizers import BertWordPieceTokenizer

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")


('ratings.txt', <http.client.HTTPMessage at 0x7fdd6cae66a0>)

In [ ]:
naver_df = pd.read_table('ratings.txt')
naver_df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [ ]:
naver_df = naver_df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(naver_df.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [ ]:

with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [ ]:
tokenizer = BertWordPieceTokenizer(lowercase=False, strip_accents=False)


lowercase : True일 경우 토크나이저는 영어의 대문자와 소문자를 동일한 문자 취급.

strip_accents : True일 경우 악센트 제거.

ex) é → e, ô → o

wordpieces_prefix : 서브워드로 쪼개졌을 경우 뒤의 서브워드에는 ##를 부착하여 원래 단어에서 분리된 것임을 표시.

ex) 안녕하세요 -> [안녕, ##하세요]

In [ ]:
data_file = 'naver_review.txt'
vocab_size = 30000
limit_alphabet = 6000
min_frequency = 5

tokenizer.train(files=data_file,
                vocab_size=vocab_size,
                limit_alphabet=limit_alphabet,
                min_frequency=min_frequency,
                )

vocab_size : 단어 집합의 크기

limit_alphabet : merge가 되지 않은 초기 토큰(character 단위)의 허용 제한 개수

min_frequency : merge가 되기 위한 pair의 최소 등장 횟수

In [ ]:

# vocab 저장
tokenizer.save_model('./')

['./vocab.txt']

In [ ]:
# vocab 로드
df = pd.read_fwf('vocab.txt', header=None)
df

,0
0,[PAD]
1,[UNK]
2,[CLS]
3,[SEP]
4,[MASK]
...,...
29995,맘을
29996,맛도
29997,망하지
29998,망한다


총 30,000개의 단어가 존재합니다. 이는 단어 집합의 크기를 30,000으로 지정하였기 때문입니다. 실제 토큰화를 수행해봅시다.

In [ ]:

encoded = tokenizer.encode('아 배고픈데 짜장면먹고싶다')
print('토큰화 결과 :',encoded.tokens)
print('정수 인코딩 :',encoded.ids)
print('디코딩 :',tokenizer.decode(encoded.ids))

토큰화 결과 : ['아', '배고', '##픈', '##데', '짜장면', '##먹고', '##싶다']
정수 인코딩 : [2111, 20631, 3776, 3382, 24680, 7872, 7379]
디코딩 : 아 배고픈데 짜장면먹고싶다


In [ ]:
encoded = tokenizer.encode('커피 한잔의 여유를 즐기다')
print('토큰화 결과 :',encoded.tokens)
print('정수 인코딩 :',encoded.ids)
print('디코딩 :',tokenizer.decode(encoded.ids))

토큰화 결과 : ['커피', '한잔', '##의', '여유', '##를', '즐기', '##다']
정수 인코딩 : [12825, 25643, 3240, 12696, 3392, 10784, 3253]
디코딩 : 커피 한잔의 여유를 즐기다


# 2. 기타 토크나이저

이 외 ByteLevelBPETokenizer, CharBPETokenizer, SentencePieceBPETokenizer 등이 존재하며 선택에 따라서 사용할 수 있습니다.

# BertWordPieceTokenizer : BERT에서 사용된 워드피스 

# 토크나이저(WordPiece Tokenizer)

# CharBPETokenizer : 오리지널 BPE

# ByteLevelBPETokenizer : BPE의 바이트 레벨 버전

# SentencePieceBPETokenizer : 앞서 본 패키지 센텐스피스(SentencePiece)와 호환되는 BPE 구현체

In [ ]:

from tokenizers import ByteLevelBPETokenizer, CharBPETokenizer, SentencePieceBPETokenizer
                            
tokenizer = SentencePieceBPETokenizer()
tokenizer.train('naver_review.txt', vocab_size=10000, min_frequency=5)

encoded = tokenizer.encode("이 영화는 정말 재미있습니다.")
print(encoded.tokens)

['▁이', '▁영화는', '▁정말', '▁재미있', '습니다.']
